In [67]:
#Final
import mysql.connector as SQLC
class DB:
#database connection and extraction of sequences and id_genebanks
    def __init__(self, password):        
        if password != "20221207":
            raise Exception("The password is wrong!")

        else:
            self.cnx = SQLC.connect(
                    host ="geo.di.uminho.pt",
                    user ="bioinformatica",
                    password = password, #"20221207"
                    database ="AP_db_KRG")
            self.sequences = []
        
    def extract_sequences(self):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT sequence FROM Gene")
        self.sequences = cursor.fetchall()
        cursor.close()
        
    def extract_ids(self):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT ID_genebank FROM Gene")
        self.sequences = cursor.fetchall()
        cursor.close()
    
    def __str__(self): 
        return self.sequences

In [61]:
#Final
import unittest
class TestDB(unittest.TestCase):
#testing DB class
    def test_setUp(self):
        self.assertTrue(DB("20221207"))
        with self.assertRaises(Exception) as error:
            self.db1 = DB("2022120")
            self.db2 = DB("20221208")
            self.db3 = DB("")
            self.db4 = DB(" ")
        self.assertEqual(error.exception.args[0], "The password is wrong!")

    def test_extract_sequences(self):
        self.db = DB("20221207")
        self.db.extract_sequences()
        self.assertIsNotNone(self.db.sequences)
        
    def test_extract_ids(self):
        self.db = DB("20221207")
        self.db.extract_ids()
        self.assertIsNotNone(self.db.sequences)
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestDB)
unittest.TextTestRunner(verbosity=3).run(suite)   

test_extract_ids (__main__.TestDB) ... ok
test_extract_sequences (__main__.TestDB) ... ok
test_setUp (__main__.TestDB) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.578s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [134]:
#FINAL
class SimpleBlast:
    def __init__(self, w, seq):
        self.w = w
        self.seq = seq
        
    def query_map(self, query):
    #returns a dictionary where key is query of length w and values are 
    #the indexes of query that match with key
        
        query_map = {}
        for i in range(len(query) - self.w + 1):
            window = query[i: i + self.w]
            if window in query_map:
                query_map[window].append(i)
            else:
                query_map[window] = [i]
        return query_map 

    def hits(self, query_map, sequence):
    #returns list of tuples where 1st index is from query and 2nd is from seq
        hits = []
        for window_query, query_offsets in query_map.items():
            if window_query in sequence:
                seq_offset = sequence.find(window_query)
                while seq_offset != -1: #because of find()
                    for query_offset in query_offsets:
                        hits.append((query_offset, seq_offset))
                    seq_offset = sequence.find(window_query, seq_offset + 1) #because while loop and find()
        return hits

    def extend_hit_dir(self, query, sequence, q_offset, s_offset, direction):
    #direction: -1 or +1
        count = 0
        matches = 0
        while q_offset >= 0 and s_offset >= 0 and q_offset < len(query) and s_offset < len(sequence):
            count += 1
            if query[q_offset] == sequence[s_offset]:
                matches += 1
            if 2 * matches < count:
                break
            q_offset += direction
            s_offset += direction
        return q_offset - direction, s_offset - direction, matches, count

    def extend_hit(self, query, seq, hit, w):
    #given a hit, will extend the search of matches to left and right    
        o1, o2 = hit
        left  = self.extend_hit_dir(query, seq, o1 - 1, o2 - 1, -1)
        right = self.extend_hit_dir(query, seq, o1 + w, o2 + w, +1)

        O1, O2, ML, SL = left
        _,   _, MR, SR = right

#         print({'ML':ML, 'SR':SR, 'SL':SL,'MR':MR})
        return O1, O2, w + SL + SR, ML + w + MR
    
    def best_hit(self, query, seq, w):
    #finds best hit from all hits and respective extensions
        query_map = self.query_map(query)
        hits = self.hits(query_map, seq)
        best_hit = None
        best_score = -1
        for hit in hits:
            o1, o2, l, m = self.extend_hit(query, seq, hit, w)
            score = m - (l - m)
            if score > best_score or (score == best_score and l < best_hit[2]):
                best_hit = (o1, o2, l, m)
                best_score = score
        return best_hit
    
# best_hit = (A,B,C,D)
# A = start position of the best match in the query
# B = starting position of the best match in the sequence
# C = best match size
# D = number of matches between nucleotides of the best match

In [31]:
class TestSBlast(unittest.TestCase):
    w = 3
    seq = 'AATATGTTATATAATAATATTT'
    query = 'AATATAT'
    sb = SimpleBlast(w,seq)
    result_qm = {'AAT': [0], 'ATA': [1, 3], 'TAT': [2, 4]}
    
    def test_querymap(self):
        self.assertEqual(self.sb.query_map(self.query),self.result_qm)
        
    def test_hits(self):
        result_hit = [(0, 0), (0, 12), (0, 15), (1, 1), (1, 8), (1, 10), (1, 13), (1, 16),
                    (3, 1), (3, 8), (3, 10), (3, 13), (3, 16), (2, 2), (2, 7), (2, 9),
                    (2, 17), (4, 2), (4, 7), (4, 9), (4, 17)]
        self.assertEqual(set(self.sb.hits(self.result_qm, self.seq)), set(result_hit))
        
    def test_extend_hit_dir(self):
        self.assertEqual(self.sb.extend_hit_dir(query, seq, 1, 16, -1), (0, 15, 2, 2))
        self.assertEqual(self.sb.extend_hit_dir(query, seq, 1, 16, 1), (6, 21, 5, 6))
        
    def test_extend_hit(self):
        self.assertEqual(self.sb.extend_hit(query, seq, (1,16), w), (0, 15, 7, 6))
    
    def test_best_hit(self):
        self.assertEqual(self.sb.best_hit(query, seq, w), (0, 0, 7, 6))  

suite = unittest.TestLoader().loadTestsFromTestCase(TestSBlast)
unittest.TextTestRunner( verbosity=3 ).run( suite )    

test_best_hit (__main__.TestSBlast) ... ok
test_extend_hit (__main__.TestSBlast) ... ok
test_extend_hit_dir (__main__.TestSBlast) ... ok
test_hits (__main__.TestSBlast) ... ok
test_querymap (__main__.TestSBlast) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.008s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [144]:
#final
class BlastResults:
    def __init__(self, query, w, password):
        self.query = query
        self.w = w
        self.password = password
        self.sequences_obj = DB(password)
        self.ids_obj = DB(password)
        self.sequences_obj.extract_sequences()
        self.ids_obj.extract_ids()
  
    def process_results(self):
    #joins the best_hit to respective id_genebank (from database)
        all_seqs = []
        for num, seq in enumerate(self.sequences_obj.sequences):
            seq_id = []
            seq = ''.join(seq)
            blast = SimpleBlast(self.w, seq)
            results = blast.best_hit(self.query, seq, self.w)
            seq_id.append(results)
            seq_id.append(''.join(self.ids_obj.sequences[num]))
            all_seqs.append(seq_id)
        return all_seqs
        
    def sort_results(self):
        def sort_key(item):
        #sorting the tuple by the last number
            if item[0]:
                return item[0][-1]
            return 0
        return sorted(self.process_results(), key=sort_key, reverse=True)
        
    def printing(self):
        # organized printing
        print(
            '{:>9} {:>15} {:>18} {:>12}  {:>5}'.format('Query position', 'Seq position', 'Length of match', 'Max match',
                                                       'Id'))
        for i in self.sort_results():
#             if i[0]:
            print(i)
#             print('{:>6} {:>18} {:>15} {:>16}      |'.format(*i[0]), i[1])
        return '_______________________________________________________________'

In [131]:
class TestRBlast(unittest.TestCase):
    query = "ACTG"
    w = 3
    tmp1_results = BlastResults("ACTGAAA", 3, "20221207")
    tmp1_results = BlastResults("ACTGAAA", 3, "20221207")
    
    def test_process_results(self):
        self.assertEqual(self.tmp1_results.process_results()[9], [, 'DS196524.1'])
    
suite = unittest.TestLoader().loadTestsFromTestCase(TestRBlast)
unittest.TextTestRunner( verbosity=3 ).run( suite )  

SyntaxError: invalid syntax (2134333747.py, line 8)

In [139]:
import unittest

class TestBlastResults(unittest.TestCase):
    def setUp(self):
        self.query = "X"
        self.w = 2
        self.password = "20221207"
        self.blast_results = BlastResults(self.query, self.w, self.password)

    def test_process_results(self):
        all_seqs = self.blast_results.process_results()
        self.assertIsNotNone(all_seqs)
        self.assertIsInstance(all_seqs, list)
        self.assertGreater(len(all_seqs), 0)
        self.assertIsInstance(all_seqs[0], list)
        self.assertEqual(len(all_seqs[0]), 2)

#     def test_sort_results(self):
#         sorted_results = self.blast_results.sort_results()
#         self.assertIsNotNone(sorted_results)
#         self.assertIsInstance(sorted_results, list)
#         self.assertGreater(len(sorted_results), 0)
#         self.assertIsInstance(sorted_results[0], list)
#         self.assertEqual(len(sorted_results[0]), 2)
#         self.assertEqual(sorted(sorted_results, key=lambda x: x[0][-1], reverse=True), sorted_results)
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestBlastResults)
unittest.TextTestRunner( verbosity=3 ).run( suite )  


test_process_results (__main__.TestBlastResults) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.479s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [145]:
query = "AAAAAAAAAAA"
w = 6
# try:
blast_results = BlastResults(query, w, "20221207")
# if blast_results == "The password is wrong":
#     print('a')
# except:
#     print("Something went wrong")
blast_results.process_results()
blast_results.printing()


Query position    Seq position    Length of match    Max match     Id
[(0, 8, 7, 6), 'DH405380.1']
[None, 'A00829.1']
[None, 'AAADK0043071.1']
[None, 'AAADS0003348.1']
[None, 'AAADW0009410.1']
[None, 'AAADX0043451.1']
[None, 'AAVX01203288.1']
[None, 'ABAAM0346030.1']
[None, 'BAAF03073813.1']
[None, 'DS196524.1']
[None, 'EC923692.1']
[None, 'EC966285.1']
[None, 'M97702.1']
[None, 'X65482.1']
[None, 'X68316.1']


'_______________________________________________________________'

In [28]:
query = "AATATAT"
seq = "AATATGTTATATAATAATATTT"
w = 3
blast_results = SimpleBlast(w, seq)
query_ma=blast_results.query_map(query)
blast_results.hits(query_ma, seq)
blast_results.extend_hit_dir(query, seq, 1, 16, 1)
blast_results.extend_hit(query, seq, (1,16), w)
blast_results.best_hit( query, seq, w)

(0, 0, 7, 6)

In [38]:
query = input("Insert your sequence query here: ")
w = input("What is the length of the query (w)? ")
pw = input("Password of the Database: ")

Insert your sequence query here: 2


In [ ]:
try:
    query = input("Insert your sequence query here: ")
    if query.isalpha():
        w = input("What is the length of the query (w)? ")
             
except:
    print("Invalid query")
#     pw = input("Password of the Database: ")
#     if pw !=